In [1]:
! pip install semantic-kernel -U

In [2]:
import os
import sys

import semantic_kernel as sk

In [3]:

kernel = sk.Kernel()

In [4]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [5]:
service_id = None

In [6]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_service(skaoai.AzureChatCompletion(deployment_name=deployment,endpoint=endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

In [7]:
from semantic_kernel.planners.basic_planner import BasicPlanner

In [8]:

planner = BasicPlanner(service_id)

In [9]:
base_skills_directory = '../../../plugins'

In [10]:
import APIPlugin.CustomAPIPlugin as custom_plugin

In [11]:

custom_plugin = kernel.import_plugin_from_object(custom_plugin.CustomAPIPlugin(), plugin_name="CustomAPI")  #.import_native_skill_from_directory(base_plugin , "APIPlugin")
writer_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "TranslatePlugin")

In [12]:
ask = """
Write email about  travellings tips based on  getting current weather in Guangzhou and translate it to Chinese"""

In [13]:
service_id

In [14]:

plan = await planner.create_plan(ask, kernel)

In [15]:
print(plan.generated_plan)

{
    "input": "travelling tips",
    "subtasks": [
        {"function": "CustomAPI.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips"},
        {"function": "EmailPlugin.WeatherMail"},
        {"function": "TranslatePlugin.MultiLanguage", "args": {"language": "Chinese"}}
    ]
}


In [16]:
results = await planner.execute_plan(plan, kernel)

In [17]:
results

'主题：我们即将出行的重要穿着提示\n\n亲爱的团队，\n\n希望这封邮件找到你们时一切都好。我们正在为即将的旅行做准备，我想分享一些重要的穿着提示，以确保我们在旅途中都能保持舒适和安全。\n\n1. 穿浅色衣服：浅色反射热量，而深色吸收热量。这可以帮助我们在炎热的天气条件下保持凉爽。\n\n2. 选择轻便宽松的衣服：宽松和轻便的衣服可以让我们的皮肤呼吸，保持凉爽。这一点尤其重要，因为我们会经常活动。\n\n3. 选择天然面料：棉、亚麻和其他天然面料透气，可以防止过热。这些材料也适合长时间旅行。\n\n4. 戴帽子：帽子或者鸭舌帽可以保护我们的头部和面部免受直射阳光的伤害。这一点非常重要，因为我们将在户外度过大部分时间。\n\n5. 使用太阳镜：保护你的眼睛免受太阳有害紫外线的伤害。这不仅是为了舒适，也是为了我们的眼睛健康。\n\n6. 涂抹防晒霜：即使我们全身都穿着衣服，最好也在暴露的皮肤部位涂抹防晒霜。这可以防止晒伤和其他皮肤损伤。\n\n请记得根据需要打包行李，并始终注意天气条件。我们在旅行中的安全和舒适非常重要。\n\n期待我们的共同旅程。\n\n顺祝商祺，\n\n[你的名字]'